<h1>Analyzing Historical Stock/Revenue Data and Building a Dashboard
 Using a Python Library </h1>
 <h2> for Tesla and GameStop <h2>
A company's stock share is a piece of the company more precisely:
<p><b>A stock (also known as equity) is a security that represents the ownership of a fraction of a corporation. This
entitles the owner of the stock to a proportion of the corporation's assets and profits equal to how much stock they own. Units of stock are called "shares." [1]</p></b>

An investor can buy a stock and sell it later. If the stock price increases, the investor profits, If it decreases,the investor with incur a loss.  Determining the stock price is complex; it depends on the number of outstanding shares, the size of the company's future profits, and much more. People trade stocks throughout the day the stock ticker is a report of the price of a certain stock, updated continuously throughout the trading session by the various stock market exchanges.


In [ ]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install nbformat==4.2.0

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

Extracting the Stock Data of Tesla using the yfinance, Ticker fn and the ticker symbol is TSLA

In [3]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [4]:
tesla_data= tesla.history(period="max")


In [5]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0

## Using Webscraping to Extract Tesla Revenue Data
Using the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm and saving
 the text of the response as a variable named `html_data`.


In [6]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text



In [7]:
soup = BeautifulSoup(html_data, "html5lib")


In [14]:
tesla_revenue= pd.read_html(url)
tesla_revenue

[    Tesla Annual Revenue (Millions of US $)  \
 0                                      2021   
 1                                      2020   
 2                                      2019   
 3                                      2018   
 4                                      2017   
 5                                      2016   
 6                                      2015   
 7                                      2014   
 8                                      2013   
 9                                      2012   
 10                                     2011   
 11                                     2010   
 12                                     2009   
 
    Tesla Annual Revenue (Millions of US $).1  
 0                                    $53,823  
 1                                    $31,536  
 2                                    $24,578  
 3                                    $21,461  
 4                                    $11,759  
 5                                    

Using the read_html function extract the table with Tesla Quarterly Revenue and store it into a dataframe named tesla_revenue, and the dataframe should have to be rename the columns Date and Revenue.



In [8]:
tesla_revenue= pd.read_html(url, match="Tesla Quarterly Revenue", flavor='bs4')[0]
tesla_revenue = tesla_revenue.rename(columns={"Tesla Quarterly Revenue (Millions of US $)":"Date", "Tesla Quarterly Revenue (Millions of US $).1":"Revenue"}, inplace=False)
tesla_revenue.head()


Date  Revenue
0  2022-09-30  $21,454
1  2022-06-30  $16,934
2  2022-03-31  $18,756
3  2021-12-31  $17,719
4  2021-09-30  $13,757

to remove an comma, dollar sign, null values, or empty strings from the Revenue column.

In [9]:
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(",","").str.replace("$","")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.head()




<ipython-input-9-bd0040dfb4b1>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(",","").str.replace("$","")


Date Revenue
0  2022-09-30   21454
1  2022-06-30   16934
2  2022-03-31   18756
3  2021-12-31   17719
4  2021-09-30   13757

In [10]:
tesla_revenue.tail()

Date Revenue
48  2010-09-30      31
49  2010-06-30      28
50  2010-03-31      21
52  2009-09-30      46
53  2009-06-30      27

Extracting the Stock Data of Gamestop using the yfinance, Ticker fn and the ticker symbol is GME


In [11]:
gamestop = yf.Ticker("GME")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [12]:
gme_data=gamestop.history(period="max")

In [13]:
gme_data.reset_index(inplace=True)
gme_data.head()

Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683251  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615921  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0

## Using Webscraping to Extract GME Revenue Data
Using the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html, and saving the text of the response as a variable named `html_data`.


In [17]:
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url).text

In [18]:
soup = BeautifulSoup(html_data, "html5lib")

In [19]:
gme_revenue = pd.read_html(url)
gme_revenue


[    GameStop Annual Revenue (Millions of US $)  \
 0                                         2020   
 1                                         2019   
 2                                         2018   
 3                                         2017   
 4                                         2016   
 5                                         2015   
 6                                         2014   
 7                                         2013   
 8                                         2012   
 9                                         2011   
 10                                        2010   
 11                                        2009   
 12                                        2008   
 13                                        2007   
 14                                        2006   
 15                                        2005   
 
    GameStop Annual Revenue (Millions of US $).1  
 0                                        $6,466  
 1                           

Using the `read_html` function extract the table with `GameStop Quarterly Revenue` and store it into a dataframe named `gme_revenue`, and the dataframe should have to be rename the columns as `Date` and `Revenue`.


In [23]:
gme_revenue= pd.read_html(url, match="GameStop Quarterly Revenue", flavor='bs4')[0]
gme_revenue = gme_revenue.rename(columns={"GameStop Quarterly Revenue (Millions of US $)":"Date", "GameStop Quarterly Revenue (Millions of US $).1":"Revenue"}, inplace=False)
gme_revenue.head()


Date Revenue
0  2020-04-30  $1,021
1  2020-01-31  $2,194
2  2019-10-31  $1,439
3  2019-07-31  $1,286
4  2019-04-30  $1,548

to remove an comma, dollar sign, null values, or empty strings from the Revenue column.

In [24]:
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(",","").str.replace("$","")
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue.head()




<ipython-input-24-b32e2cf50e3f>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(",","").str.replace("$","")


Date Revenue
0  2020-04-30    1021
1  2020-01-31    2194
2  2019-10-31    1439
3  2019-07-31    1286
4  2019-04-30    1548

In [ ]:
gme_revenue.tail()

Date Revenue
57  2006-01-31    1667
58  2005-10-31     534
59  2005-07-31     416
60  2005-04-30     475
61  2005-01-31     709

## Plot Tesla Stock Graph


Use the `make_graph` function to graph the Tesla Stock Data


In [ ]:
make_graph(tesla_data, tesla_revenue, "Tesla")

## Plot GameStop Stock Graph


Using the `make_graph` function to graph the GameStop Stock Data


In [ ]:
make_graph(gme_data, gme_revenue, "GameStop")

## DONE BY : P.K SOLLARASU